In [1]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()
import re

## Data
**Rolling stone's article: [The 500 Greatest Albums of All Time](https://www.rollingstone.com/music/music-lists/best-albums-of-all-time-1062063/arcade-fire-%ef%bb%bffuneral-1062733/)**
- csv from music database [musicbrainz](https://musicbrainz.org/series/6a4b53b9-2756-4afe-93f2-306039d41910)

In [2]:
stone = pd.read_excel("../data/musicbrainzset.xlsx")

In [3]:
stone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Number    500 non-null    int64  
 1   Year      500 non-null    int64  
 2   Album     500 non-null    object 
 3   Artist    500 non-null    object 
 4   Type      500 non-null    object 
 5   Rating    462 non-null    float64
 6   Releases  500 non-null    int64  
 7   Genre     500 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 31.4+ KB


In [4]:
stone.isna().sum()

Number       0
Year         0
Album        0
Artist       0
Type         0
Rating      38
Releases     0
Genre        0
dtype: int64

### Data cleansing

In [5]:
#como hay nombres de álbumes que son números, los convertimos a string.
def num_string(dato):
    if type(dato)== int:
        return str(dato)
    else:
        return dato

In [6]:
stone["Album"] = stone["Album"].apply(num_string)

In [7]:
stone.Artist = stone.Artist.str.strip()
stone.Album = stone.Album.str.strip()
stone.Type = stone.Type.str.strip()
stone.Genre = stone.Genre.str.strip()

In [8]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]
stone["Subgenre"] = stone["Genre"].str.extract(r",\s*(.*$)")[0]

In [9]:
stone.Gen = stone.Gen.str.strip()
stone.Subgenre = stone.Subgenre.str.strip()

In [10]:
stone.Rating.fillna(0,inplace = True)
#no rellenamos los nans de rating para que luego no estropeen la media y sean considerada float
stone.Subgenre.fillna("None",inplace = True)

In [11]:
#hay algunos títulos de album que hay que cambiar de nombre
stone.Album = stone.Album.str.replace(r'(Metallica...The Black Album..)',"Metallica",regex=True)
stone.Album = stone.Album.str.replace(r'(Eagles...st album.)',"Eagles",regex=True)
stone.Artist = stone.Artist.str.replace("Neil Young with Crazy Horse","Neil Young & Crazy Horse",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Bob Dylan...The Band)","Bob Dylan And The Band",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Prince\s+.*)","Prince & The Revolution",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Rufus...Chaka Khan)","Rufus",regex=True)
stone.Album = stone.Album.str.replace(r'(Proud Mary.*)',"Best Of / Proud Mary",regex=True)
stone.Artist = stone.Artist.str.replace(r'(.*The Velvet Underground.*)',"The Velvet Underground",regex= True)
stone.Artist = stone.Artist.str.replace("‐","-",regex = True)
stone.Album = stone.Album.str.replace("‐","-",regex = True)
stone.Gen = stone.Gen.str.replace("piunk","punk",regex = True)
stone.Gen = stone.Gen.str.replace("punk rock","punk",regex = True)
stone.Gen = stone.Gen.str.replace("folk rock","folk",regex = True)

In [12]:
#creamos nuevas columnas que haremos compatibles con url
stone["Alb_url"] = stone["Album"]
stone["Art_url"] = stone["Artist"]

In [13]:
#algunos nombres de discos hay que cambiarlos
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Band...The Brown Album..)',"The Band",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^Sign.*the Times$)',"Sign 'O' The Times",regex=True)

In [14]:
#función que cambia los caracteres especiales para las urls
def url_prep(df, colum):
    url_replace = {"'":"%27",",":"%2c", "&":"%26",".":"%2e","/":"%2f","#":"%23","(":"%28",")":"%29","-":"%2d",'"':"%22",r"(\s+)":"%20" }
    for key,value in url_replace.items():
        df[colum] = df[colum].str.replace(key,value,regex=True)

In [15]:
df = stone
colum = "Alb_url"
url_prep(df,colum)
colum = "Art_url"
url_prep(df,colum)

In [16]:
stone.sample(3)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
121,122,1994,The Downward Spiral,Nine Inch Nails,Album,3.05,32,rock,rock,None,The%20Downward%20Spiral,Nine%20Inch%20Nails
126,127,1962,Modern Sounds in Country and Western Music,Ray Charles,Album,4.50,5,"country , soul",country,soul,Modern%20Sounds%20in%20Country%20and%20Western...,Ray%20Charles
118,119,1969,Stand!,Sly & the Family Stone,Album,4.00,12,funk,funk,None,Stand!,Sly%20%26%20the%20Family%20Stone


# Principal enrichment

- Enriquecimiento con los datos datos de reproducciones y usuarios de la red social musical [lastfm](https://www.last.fm/es/)

In [17]:
apikey = os.getenv("apikey") #apikey de lastfm

In [18]:
#comprobamos respuestas de las urls
def urls_llamadas (df,art,alb):
    api_urls = []
    apikey = os.getenv("apikey")
    for s in range(len(df)):
        artist = df.loc[s,art]
        album = df.loc[s,alb]
        api_urls.append(f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={apikey}&artist={artist}&album={album}&format=json")
    request_dic = [] 
    i = 0
    for a in api_urls:
        res = requests.get(a).json()
        request_dic.append(res["album"])
        n = len(api_urls)/10
        if i%n == 0: 
            print(f"{i} done")
        elif i == (len(api_urls)-1):
            print(f"{i} done,finished")
        i+=1
    return request_dic

In [19]:
df = stone
art = "Art_url"
alb = "Alb_url"
Top500_last = urls_llamadas (df,art,alb)

0 done
50 done
100 done
150 done
200 done
250 done
300 done
350 done
400 done
450 done
499 done,finished


In [20]:
stone.to_csv("../data/stone.csv",index=False)

In [21]:
Top500_last_df = pd.DataFrame(Top500_last)

In [22]:
Top500_last_df.head(3)

,artist,mbid,tags,playcount,image,tracks,url,name,listeners,wiki
0,Marvin Gaye,0f2ab0b8-99a3-41e6-a07a-c2f506bd5e34,{'tag': [{'url': 'https://www.last.fm/tag/soul...,5258671,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/Marvin+Gaye/What%27s...,What's Going On,546458,"{'published': '08 Aug 2012, 04:25', 'summary':..."
1,The Beach Boys,05b2f6c8-f9fd-4499-bd70-6f7d629a9cd4,{'tag': [{'url': 'https://www.last.fm/tag/60s'...,13411664,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/The+Beach+Boys/Pet+S...,Pet Sounds,543630,"{'published': '07 Jul 2017, 12:48', 'summary':..."
2,Joni Mitchell,008a8b2e-3b20-3d77-999b-4f27378bf868,{'tag': [{'url': 'https://www.last.fm/tag/folk...,9405283,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/Joni+Mitchell/Blue,Blue,537629,"{'published': '08 Aug 2021, 14:19', 'summary':..."


In [23]:
Top500_last_df.to_csv("../data/otras/Top500LastFM_tracks.csv",index=False) #meloguardoporsiacaso

In [24]:
Top500_last_df.sample()

,artist,mbid,tags,playcount,image,tracks,url,name,listeners,wiki
205,David Bowie,02e2a2f6-0da2-34cb-8e65-a05447ed1994,{'tag': [{'url': 'https://www.last.fm/tag/art+...,7514418,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/David+Bowie/Low,Low,324794,"{'published': '06 Jun 2015, 17:28', 'summary':..."


In [25]:
Top500_last_df[["playcount","listeners"]] = Top500_last_df[["playcount","listeners"]].astype("int64")

In [26]:
Lastfm_500 = Top500_last_df[["artist","playcount","name","listeners"]]
Lastfm_500.head(3)

,artist,playcount,name,listeners
0,Marvin Gaye,5258671,What's Going On,546458
1,The Beach Boys,13411664,Pet Sounds,543630
2,Joni Mitchell,9405283,Blue,537629


In [27]:
stone_500 = stone[["Number","Year","Album","Artist","Type","Rating","Gen"]]
stone_500.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen
439,440,1970,Coal Miner's Daughter,Loretta Lynn,Album,0.00,country
61,62,1987,Appetite for Destruction,Guns N' Roses,Album,4.45,rock
19,20,2000,Kid A,Radiohead,Album,4.60,rock


In [28]:
stone_500_richment = stone_500.merge(Lastfm_500,left_index=True, right_index=True)
stone_500_richment.head(3)

,Number,Year,Album,Artist,Type,Rating,Gen,artist,playcount,name,listeners
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,r&b,Marvin Gaye,5258671,What's Going On,546458
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,rock,The Beach Boys,13411664,Pet Sounds,543630
2,3,1971,Blue,Joni Mitchell,Album,4.45,folk,Joni Mitchell,9405283,Blue,537629


In [29]:
stone_500_rich =stone_500_richment [["Number","Year","Album","Artist","Type","Rating","Gen","playcount","listeners"]]
stone_500_rich.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
463,464,1973,3+3,The Isley Brothers,Album,4.00,funk,1061,495
87,88,1971,Hunky Dory,David Bowie,Album,4.35,rock,10384681,516263
31,32,2016,Lemonade,Beyoncé,Album,4.10,pop,21989100,463907


In [30]:
stone_500_rich.sort_values("listeners").head(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
464,465,2003,The Best of the Classic Years,King Sunny Adé,Album + Compilation,0.0,juju,1574,115
488,489,1991,Back to Mono (1958–1969),Phil Spector,Album + Compilation,0.0,pop,5594,184
404,405,1972,Nuggets: Original Artyfacts From the First Psy...,Various Artists,Album + Compilation,0.0,rock,7364,245


In [31]:
stone_500_rich.to_csv("../data/stone_500_rich.csv",index=False)